# Movie Recommendation System

### Import all dependencies

In [45]:
import numpy as np
import pandas as pd
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Preprocessing

In [46]:
# load dataset (download the dataset from https://kaggle.com/datasets/muhammetgamal5/tmdbmoviescsv)
data = pd.read_csv("tmdb-movies.csv")

In [47]:
# show dataset
data.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


In [48]:
# number of rows and columns
print("Shape of the dataset:", data.shape)

Shape of the dataset: (10866, 21)


In [49]:
# missing values
print("Missing values per column:\n", data.isnull().sum())

Missing values per column:
 id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64


In [50]:
# Potential Features in the Dataset:
# id, imdb_id, popularity, budget, revenue, original_title,
# cast, homepage, director, tagline, keywords, overview, 
# runtime, genres, production_companies, release_date, 
# vote_count, vote_average, release_year, budget_adj, revenue_adj
#
# After evaluation, we will use:
# 'original_title', 'genres', 'keywords', 'tagline', 'cast', 'director', 'overview'
selected_features = ['original_title', 'genres', 'keywords', 'tagline', 'cast', 'director', 'overview']
data = data[selected_features]
data.head()

,original_title,genres,keywords,tagline,cast,director,overview
0,Jurassic World,Action|Adventure|Science Fiction|Thriller,monster|dna|tyrannosaurus rex|velociraptor|island,The park is open.,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,Twenty-two years after the events of Jurassic ...
1,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller,future|chase|post-apocalyptic|dystopia|australia,What a Lovely Day.,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,An apocalyptic story set in the furthest reach...
2,Insurgent,Adventure|Science Fiction|Thriller,based on novel|revolution|dystopia|sequel|dyst...,One Choice Can Destroy You,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,Beatrice Prior must confront her inner demons ...
3,Star Wars: The Force Awakens,Action|Adventure|Science Fiction|Fantasy,android|spaceship|jedi|space opera|3d,Every generation has a story.,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Thirty years after defeating the Galactic Empi...
4,Furious 7,Action|Crime|Thriller,car race|speed|revenge|suspense|car,Vengeance Hits Home,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Deckard Shaw seeks revenge against Dominic Tor...


In [51]:
# replacing null values
for feature in selected_features:
    data[feature] = data[feature].fillna('')

In [52]:
# combine all the selected features
data['combined_features'] = data['genres'] + ' ' + data['keywords'] + ' ' + data['tagline'] + ' ' + data['cast'] + ' ' + data['director'] + ' ' + data['overview']

In [53]:
# Show combined features
data[['original_title', 'combined_features']].head()

,original_title,combined_features
0,Jurassic World,Action|Adventure|Science Fiction|Thriller mons...
1,Mad Max: Fury Road,Action|Adventure|Science Fiction|Thriller futu...
2,Insurgent,Adventure|Science Fiction|Thriller based on no...
3,Star Wars: The Force Awakens,Action|Adventure|Science Fiction|Fantasy andro...
4,Furious 7,Action|Crime|Thriller car race|speed|revenge|s...


### Feature Extraction

In [54]:
# perform feature extraction

In [55]:
# converting to numerical values
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(data['combined_features'])

In [56]:
#print feature vectors
print(feature_vectors)

  (np.int32(0), np.int32(820))	0.05242389129226209
  (np.int32(0), np.int32(1014))	0.0612464834689844
  (np.int32(0), np.int32(37875))	0.06649923117571706
  (np.int32(0), np.int32(15559))	0.06659941635011578
  (np.int32(0), np.int32(42903))	0.04873600831207453
  (np.int32(0), np.int32(28745))	0.10996054276352246
  (np.int32(0), np.int32(12435))	0.1552290296890786
  (np.int32(0), np.int32(44181))	0.1632447364825671
  (np.int32(0), np.int32(36010))	0.14703851636957438
  (np.int32(0), np.int32(45161))	0.18814068177590537
  (np.int32(0), np.int32(21900))	0.10378646283040217
  (np.int32(0), np.int32(42724))	0.04700903946401648
  (np.int32(0), np.int32(31796))	0.35616420217525824
  (np.int32(0), np.int32(21860))	0.03533035180653477
  (np.int32(0), np.int32(30940))	0.12594725624220293
  (np.int32(0), np.int32(8185))	0.08706172972048314
  (np.int32(0), np.int32(33487))	0.15029453861851236
  (np.int32(0), np.int32(6308))	0.1552290296890786
  (np.int32(0), np.int32(10639))	0.14115143860557566
  

### Cosine Similarity

In [57]:
#calculate cosine similarity of feature vectors
similarity = cosine_similarity(feature_vectors)

In [58]:
# show/print the results
print("Similarity matrix shape:", similarity.shape)

Similarity matrix shape: (10866, 10866)


In [59]:
# input movie name 
movie_name = "Spider Man"

In [60]:
# creating a list with all the movie names
list_of_all_titles = data['original_title'].tolist()

In [61]:
#print movie list
print(list_of_all_titles)

['Jurassic World', 'Mad Max: Fury Road', 'Insurgent', 'Star Wars: The Force Awakens', 'Furious 7', 'The Revenant', 'Terminator Genisys', 'The Martian', 'Minions', 'Inside Out', 'Spectre', 'Jupiter Ascending', 'Ex Machina', 'Pixels', 'Avengers: Age of Ultron', 'The Hateful Eight', 'Taken 3', 'Ant-Man', 'Cinderella', 'The Hunger Games: Mockingjay - Part 2', 'Tomorrowland', 'Southpaw', 'San Andreas', 'Fifty Shades of Grey', 'The Big Short', 'Mission: Impossible - Rogue Nation', 'Ted 2', 'Kingsman: The Secret Service', 'Spotlight', 'Maze Runner: The Scorch Trials', 'Mr. Holmes', 'Chappie', 'Pitch Perfect 2', 'Bridge of Spies', 'Goosebumps', 'Room', 'Solace', 'The Good Dinosaur', 'Run All Night', 'Brooklyn', 'Straight Outta Compton', 'The Last Witch Hunter', 'Steve Jobs', 'The Man from U.N.C.L.E.', 'The Age of Adaline', 'Hardcore Henry', 'Home', 'Paper Towns', 'Wild Card', 'The Lobster', 'Carol', 'The Intern', 'Everest', 'Joy', 'Point Break', 'Burnt', 'Self/less', 'Mortdecai', 'The Danish G

In [62]:
# finding the close match with input
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
if find_close_match:
    close_match = find_close_match[0]
else:
    print("No close matches found for the given movie.")
    exit()


In [63]:
#print matched list
match_list = data[data['original_title'] == close_match]
print(movie_name)

Spider Man


In [64]:
# the movie name itself
movie_name = data[data['original_title'] == close_match]
print(movie_name)


     original_title          genres  \
3916     Spider-Man  Fantasy|Action   

                                               keywords  \
3916  loss of lover|spider|thanksgiving|bad boss|hos...   

                                           tagline  \
3916  With great power comes great responsibility.   

                                                   cast   director  \
3916  Tobey Maguire|Willem Dafoe|Kirsten Dunst|James...  Sam Raimi   

                                               overview  \
3916  After being bitten by a genetically altered sp...   

                                      combined_features  
3916  Fantasy|Action loss of lover|spider|thanksgivi...  


In [65]:
# finding index in dataset
index_of_movie = data[data['original_title'] == close_match].index[0]
print(index_of_movie)


3916


In [66]:
# calculate similarity score and display similarity list
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, np.float64(0.012695110649098805)), (1, np.float64(0.025470949033480385)), (2, np.float64(0.007320991550536917)), (3, np.float64(0.0173755323620456)), (4, np.float64(0.012774274102921859)), (5, np.float64(0.0059954376641679534)), (6, np.float64(0.008549145930413782)), (7, np.float64(0.012845645463046867)), (8, np.float64(0.01770270612832659)), (9, np.float64(0.008870697799477766)), (10, np.float64(0.021515693593800428)), (11, np.float64(0.010784050261047881)), (12, np.float64(0.005806906324633915)), (13, np.float64(0.022954761802751907)), (14, np.float64(0.007871672096888755)), (15, np.float64(0.01155325221232424)), (16, np.float64(0.010577637468910349)), (17, np.float64(0.006638147740067415)), (18, np.float64(0.020151631083914812)), (19, np.float64(0.013686130586080071)), (20, np.float64(0.036906415993628414)), (21, np.float64(0.029332695013446413)), (22, np.float64(0.005503653089701188)), (23, np.float64(0.010524977141216143)), (24, np.float64(0.0022139799800930705)), (25, np.flo

In [67]:
# sort this list to have the highest similarity score
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)


In [68]:
# code to print similar movies
print("Movies recommended for you if you liked '{}':".format(close_match))
i = 1
for movie in sorted_similar_movies[1:11]:
    index = movie[0]
    title_from_index = data['original_title'].iloc[index]
    print(i, '.', title_from_index)
    i += 1

Movies recommended for you if you liked 'Spider-Man':
1 . Spider-Man 2
2 . Spider-Man 3
3 . Spider
4 . Charlotte's Web
5 . The Amazing Spider-Man 2
6 . The Amazing Spider-Man
7 . Big Ass Spider!
8 . Brothers
9 . Oz: The Great and Powerful
10 . Charlotte's Web


### Recommandation System

In [ ]:
# take movie name
movie_name = input("Enter a movie name: ")

# finding the close match with input
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

# closest match - the searched movie most of the time
if len(find_close_match) == 0:
    print("No close matches found for the given movie.")
else:
    close_match = find_close_match[0]  # Select the best match

    # index of closest match
    index_of_movie = data[data['original_title'] == close_match].index[0]

    # make a list with similarity score and index, of that movie
    similarity_score = list(enumerate(similarity[index_of_movie]))

    # sort to get most similar movies at first
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # print the results
    print("\nMovies recommended for you if you liked '{}':".format(close_match))
    i = 1
    for movie in sorted_similar_movies[1:11]:  # Exclude the first one since it's the same movie
        index = movie[0]
        title_from_index = data['original_title'].iloc[index]
        print(i, '.', title_from_index)
        i += 1
